In [1]:
import numpy as np
import tensorflow as tf
import collections

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def build_dataset(words, n_words):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [3]:
with open('from.txt', 'r') as fopen:
    text_from = fopen.read().lower().split('\n')
with open('to.txt', 'r') as fopen:
    text_to = fopen.read().lower().split('\n')
print('len from: %d, len to: %d'%(len(text_from), len(text_to)))

len from: 265, len to: 265


In [4]:
concat_from = ' '.join(text_from).split()
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])

vocab from size: 331
Most common words [('you', 73), ('is', 67), ('what', 63), ('a', 49), ('the', 40), ('do', 36)]
Sample data [88, 55, 166, 55, 264, 55, 171, 55, 282, 12] ['hi', 'good', 'morning', 'good', 'afternoon', 'good', 'evening', 'good', 'night', 'how']


In [5]:
concat_to = ' '.join(text_to).split()
vocabulary_size_to = len(list(set(concat_to)))
data_to, count_to, dictionary_to, rev_dictionary_to = build_dataset(concat_to, vocabulary_size_to)
print('vocab to size: %d'%(vocabulary_size_to))
print('Most common words', count_to[4:10])
print('Sample data', data_to[:10], [rev_dictionary_to[i] for i in data_to[:10]])

vocab to size: 504
Most common words [('i', 127), ('you', 55), ('a', 47), ('to', 44), ('the', 40), ('it', 38)]
Sample data [144, 123, 36, 405, 36, 452, 36, 245, 36, 468] ['hi', 'there', 'good', 'morning', 'good', 'afternoon', 'good', 'evening', 'good', 'night']


In [6]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [7]:
class Chatbot:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, to_dict_size, batch_size,
                 grad_clip=5.0, beam_width=5, force_teaching_ratio=0.5):
        
        def cells(size, reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size, initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.placeholder(tf.int32, [None])
        self.Y_seq_len = tf.placeholder(tf.int32, [None])
        
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embeddings = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        def bahdanau(size):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size, 
                                                                    memory = encoder_embedded)
            return tf.contrib.seq2seq.AttentionWrapper(cell = cells(size), 
                                                        attention_mechanism = attention_mechanism,
                                                        attention_layer_size = size)
        
        def luong(size):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size, 
                                                                    memory = encoder_embedded)
            return tf.contrib.seq2seq.AttentionWrapper(cell = cells(size), 
                                                        attention_mechanism = attention_mechanism,
                                                        attention_layer_size = size)
        
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = bahdanau(size_layer//2),
                cell_bw = luong(size_layer//2),
                inputs = encoder_embedded,
                sequence_length = self.X_seq_len,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_%d'%(n))
            encoder_embedded = tf.concat((out_fw, out_bw), 2)
            
        bi_state = tf.concat((state_fw[0],state_bw[0]), -1)
        encoder_state = tuple([bi_state] * num_layers)
        
        dense = tf.layers.Dense(to_dict_size)
        
        attention_mechanism = tf.contrib.seq2seq.LuongAttention(
            num_units = size_layer, 
            memory = encoder_embedded,
            memory_sequence_length = self.X_seq_len)
        luong_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
            num_units = size_layer, 
            memory = encoder_embedded,
            memory_sequence_length = self.X_seq_len)
        bahdanau_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        decoder_cells = tf.nn.rnn_cell.MultiRNNCell([luong_cells, bahdanau_cells])
        
        with tf.variable_scope('decode'):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                num_units = size_layer, 
                memory = encoder_embedded,
                memory_sequence_length = self.X_seq_len)
            luong_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_embedded,
                memory_sequence_length = self.X_seq_len)
            bahdanau_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            decoder_cells = tf.nn.rnn_cell.MultiRNNCell([luong_cells, bahdanau_cells])
            main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
            decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
            training_helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
            inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
                sequence_length = self.Y_seq_len,
                embedding = decoder_embeddings,
                sampling_probability = 1 - force_teaching_ratio,
                time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = decoder_cells,
                helper = training_helper,
                initial_state = decoder_cells.zero_state(batch_size, tf.float32),
                output_layer = tf.layers.Dense(to_dict_size))
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_embedded, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(
                num_units = size_layer, 
                memory = encoder_out_tiled,
                memory_sequence_length = X_seq_len_tiled)
            luong_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer,reuse=True) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units = size_layer, 
                memory = encoder_out_tiled,
                memory_sequence_length = X_seq_len_tiled)
            bahdanau_cells = tf.contrib.seq2seq.AttentionWrapper(
                cell = tf.nn.rnn_cell.MultiRNNCell([cells(size_layer,reuse=True) for _ in range(num_layers)]),
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
            decoder_cells = tf.nn.rnn_cell.MultiRNNCell([luong_cells, bahdanau_cells])
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cells,
                embedding = decoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = decoder_cells.zero_state(batch_size * beam_width, tf.float32),
                beam_width = beam_width,
                output_layer = tf.layers.Dense(to_dict_size, _reuse=True),
                length_penalty_weight = 0.0)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.predicting_ids = predicting_decoder_output.predicted_ids[:, :, 0]
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.logits,
                                                     targets = self.Y,
                                                     weights = masks)
        params = tf.trainable_variables()
        gradients = tf.gradients(self.cost, params)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, grad_clip)
        self.optimizer = tf.train.AdamOptimizer().apply_gradients(zip(clipped_gradients, params))

In [8]:
size_layer = 128
num_layers = 2
embedded_size = 128
learning_rate = 0.001
batch_size = 32
epoch = 50

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Chatbot(size_layer, num_layers, embedded_size, vocabulary_size_from + 4, 
                vocabulary_size_to + 4, batch_size)
sess.run(tf.global_variables_initializer())

In [10]:
def str_idx(corpus, dic):
    X = []
    for i in corpus:
        ints = []
        for k in i.split():
            try:
                ints.append(dic[k])
            except Exception as e:
                print(e)
                ints.append(2)
        X.append(ints)
    return X

In [11]:
X = str_idx(text_from, dictionary_from)
Y = str_idx(text_to, dictionary_to)

'mexican'
'says'


In [12]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

def check_accuracy(logits, Y):
    acc = 0
    for i in range(logits.shape[0]):
        internal_acc = 0
        for k in range(len(Y[i])):
            try:
                if Y[i][k] == logits[i][k]:
                    internal_acc += 1
            except:
                continue
        acc += (internal_acc / len(Y[i]))
    return acc / logits.shape[0]

In [13]:
for i in range(epoch):
    total_loss, total_accuracy = 0, 0
    for k in range(0, (len(text_from) // batch_size) * batch_size, batch_size):
        batch_x, seq_x = pad_sentence_batch(X[k: k+batch_size], PAD)
        batch_y, seq_y = pad_sentence_batch(Y[k: k+batch_size], PAD)
        predicted, loss, _ = sess.run([model.predicting_ids, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                model.Y:batch_y,
                                                model.X_seq_len:seq_x,
                                                model.Y_seq_len:seq_y})
        total_loss += loss
        total_accuracy += check_accuracy(predicted,batch_y)
    total_loss /= (len(text_from) // batch_size)
    total_accuracy /= (len(text_from) // batch_size)
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(i+1, total_loss, total_accuracy))

epoch: 1, avg loss: 6.161469, avg accuracy: 0.015566
epoch: 2, avg loss: 5.714868, avg accuracy: 0.025816
epoch: 3, avg loss: 5.427912, avg accuracy: 0.021140
epoch: 4, avg loss: 5.245713, avg accuracy: 0.031671
epoch: 5, avg loss: 5.119965, avg accuracy: 0.035701
epoch: 6, avg loss: 4.972153, avg accuracy: 0.034472
epoch: 7, avg loss: 4.801816, avg accuracy: 0.036430
epoch: 8, avg loss: 4.591227, avg accuracy: 0.040975
epoch: 9, avg loss: 4.353438, avg accuracy: 0.049351
epoch: 10, avg loss: 4.099843, avg accuracy: 0.053715
epoch: 11, avg loss: 3.855474, avg accuracy: 0.060539
epoch: 12, avg loss: 3.651207, avg accuracy: 0.062141
epoch: 13, avg loss: 3.512224, avg accuracy: 0.066831
epoch: 14, avg loss: 3.318720, avg accuracy: 0.072654
epoch: 15, avg loss: 3.117723, avg accuracy: 0.080067
epoch: 16, avg loss: 2.942628, avg accuracy: 0.086500
epoch: 17, avg loss: 2.794498, avg accuracy: 0.103053
epoch: 18, avg loss: 2.627144, avg accuracy: 0.102751
epoch: 19, avg loss: 2.499072, avg ac